**1. Импортирование необходимых библиотек.**

In [130]:
import pandas as pd
import numpy as np
from feature_engine.creation import MathFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans, SpectralClustering, AgglomerativeClustering, BisectingKMeans
from tabgan.sampler import OriginalGenerator, GANGenerator, ForestDiffusionGenerator, LLMGenerator

**2. Предобработка данных.**

**2.1. Загрузка данных.**

In [131]:
filename="data.csv"
filepath="./"+filename
df_features = pd.read_csv(filepath, delimiter=";")

**2.2. Просмотр данных.**

In [132]:
df_features.head()

,Пол,Приоритет,Балл по математике,Балл по русскому языку,Балл по выбранному предмету,Балл за индивидуальные достижения,Наличие серебрянной медали,Наличие золотой медали
0,1,1,95,67,72,20,0,0
1,0,1,64,67,64,0,0,0
2,0,1,58,69,56,0,0,0
3,0,1,39,58,68,0,0,0
4,0,1,46,52,53,0,0,0


In [133]:
df_features.describe()

,Пол,Приоритет,Балл по математике,Балл по русскому языку,Балл по выбранному предмету,Балл за индивидуальные достижения,Наличие серебрянной медали,Наличие золотой медали
count,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000
mean,0.206897,1.258621,69.732759,71.017241,68.620690,4.948276,0.025862,0.017241
std,0.406838,0.512805,15.240311,13.480409,12.522436,6.220366,0.159412,0.130734
min,0.000000,1.000000,0.000000,0.000000,18.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,64.000000,64.000000,62.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,74.000000,72.000000,71.500000,3.000000,0.000000,0.000000
75%,0.000000,1.000000,80.000000,80.250000,78.000000,10.000000,0.000000,0.000000
max,1.000000,3.000000,95.000000,100.000000,94.000000,30.000000,1.000000,1.000000


In [134]:
df_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 8 columns):
 #   Column                             Non-Null Count  Dtype
---  ------                             --------------  -----
 0   Пол                                116 non-null    int64
 1   Приоритет                          116 non-null    int64
 2   Балл по математике                 116 non-null    int64
 3   Балл по русскому языку             116 non-null    int64
 4   Балл по выбранному предмету        116 non-null    int64
 5   Балл за индивидуальные достижения  116 non-null    int64
 6   Наличие серебрянной медали         116 non-null    int64
 7   Наличие золотой медали             116 non-null    int64
dtypes: int64(8)
memory usage: 7.4 KB


In [135]:
df_features.isnull().sum()

Пол                                  0
Приоритет                            0
Балл по математике                   0
Балл по русскому языку               0
Балл по выбранному предмету          0
Балл за индивидуальные достижения    0
Наличие серебрянной медали           0
Наличие золотой медали               0
dtype: int64

**2.3. Генерация новых признаков.**

**2.3.1. Средний балл по экзаменам.**

In [136]:
mf_mean = MathFeatures(variables=["Балл по математике", "Балл по русскому языку", "Балл по выбранному предмету"], func = "mean", new_variables_names=["Средний балл по экзаменам"])
mf_mean.fit(df_features)
df_features = mf_mean.transform(df_features)
df_features

,Пол,Приоритет,Балл по математике,Балл по русскому языку,Балл по выбранному предмету,Балл за индивидуальные достижения,Наличие серебрянной медали,Наличие золотой медали,Средний балл по экзаменам
0,1,1,95,67,72,20,0,0,78.000000
1,0,1,64,67,64,0,0,0,65.000000
2,0,1,58,69,56,0,0,0,61.000000
3,0,1,39,58,68,0,0,0,55.000000
4,0,1,46,52,53,0,0,0,50.333333
...,...,...,...,...,...,...,...,...,...
111,0,1,72,75,80,0,0,0,75.666667
112,0,1,70,61,62,0,0,0,64.333333
113,1,1,70,60,53,4,0,0,61.000000
114,0,1,52,46,46,0,0,0,48.000000


**2.3.2. Сумма баллов.**

In [137]:
mf_sum = MathFeatures(variables=["Балл по математике", "Балл по русскому языку", "Балл по выбранному предмету", "Балл за индивидуальные достижения"], func = "sum", new_variables_names=["Сумма баллов"])
mf_sum.fit(df_features)
df_features = mf_sum.transform(df_features)
df_features

,Пол,Приоритет,Балл по математике,Балл по русскому языку,Балл по выбранному предмету,Балл за индивидуальные достижения,Наличие серебрянной медали,Наличие золотой медали,Средний балл по экзаменам,Сумма баллов
0,1,1,95,67,72,20,0,0,78.000000,254
1,0,1,64,67,64,0,0,0,65.000000,195
2,0,1,58,69,56,0,0,0,61.000000,183
3,0,1,39,58,68,0,0,0,55.000000,165
4,0,1,46,52,53,0,0,0,50.333333,151
...,...,...,...,...,...,...,...,...,...,...
111,0,1,72,75,80,0,0,0,75.666667,227
112,0,1,70,61,62,0,0,0,64.333333,193
113,1,1,70,60,53,4,0,0,61.000000,187
114,0,1,52,46,46,0,0,0,48.000000,144


**2.3.3. Процент от максимального балла.**

In [138]:
mf_percent = MathFeatures(variables=["Балл по математике", "Балл по русскому языку", "Балл по выбранному предмету", "Балл за индивидуальные достижения"], func = lambda scores: (sum(scores) / 347) * 100, new_variables_names=["Процент от маскимального балла"])
mf_percent.fit(df_features)
df_features = mf_percent.transform(df_features)
df_features

,Пол,Приоритет,Балл по математике,Балл по русскому языку,Балл по выбранному предмету,Балл за индивидуальные достижения,Наличие серебрянной медали,Наличие золотой медали,Средний балл по экзаменам,Сумма баллов,Процент от маскимального балла
0,1,1,95,67,72,20,0,0,78.000000,254,73.198847
1,0,1,64,67,64,0,0,0,65.000000,195,56.195965
2,0,1,58,69,56,0,0,0,61.000000,183,52.737752
3,0,1,39,58,68,0,0,0,55.000000,165,47.550432
4,0,1,46,52,53,0,0,0,50.333333,151,43.515850
...,...,...,...,...,...,...,...,...,...,...,...
111,0,1,72,75,80,0,0,0,75.666667,227,65.417867
112,0,1,70,61,62,0,0,0,64.333333,193,55.619597
113,1,1,70,60,53,4,0,0,61.000000,187,53.890490
114,0,1,52,46,46,0,0,0,48.000000,144,41.498559


**2.4. Генерация новых данных.**

**2.4.1. Кластеризация имеющихся данных.**

Попробуем кластеризовать имеющиеся данные с помощью разных методов из библиотеки sklearn

**2.4.1.1. Метод KMeans.**

In [139]:
kmeans = KMeans(n_clusters=4, random_state=0, n_init="auto").fit(df_features)
kmeans.labels_

array([2, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       3, 0, 0, 1, 1, 1], dtype=int32)

In [140]:
df_kmeans_target = pd.DataFrame(kmeans.labels_, columns=["Успеваемость"])
df_kmeans_target.value_counts()

Успеваемость
0               63
2               28
1               24
3                1
Name: count, dtype: int64

In [141]:
kmeans_clustering_data = pd.concat([df_features, df_kmeans_target], axis=1)

In [142]:
kmeans_clustering_data[kmeans_clustering_data["Успеваемость"] == 0]

,Пол,Приоритет,Балл по математике,Балл по русскому языку,Балл по выбранному предмету,Балл за индивидуальные достижения,Наличие серебрянной медали,Наличие золотой медали,Средний балл по экзаменам,Сумма баллов,Процент от маскимального балла,Успеваемость
1,0,1,64,67,64,0,0,0,65.000000,195,56.195965,0
32,0,1,70,83,78,5,0,0,77.000000,236,68.011527,0
33,0,1,82,67,86,0,0,0,78.333333,235,67.723343,0
34,0,1,76,81,78,0,0,0,78.333333,235,67.723343,0
35,0,1,84,73,77,0,0,0,78.000000,234,67.435159,0
...,...,...,...,...,...,...,...,...,...,...,...,...
91,0,1,58,94,62,0,0,0,71.333333,214,61.671470,0
92,1,1,64,64,64,5,0,0,64.000000,197,56.772334,0
93,0,1,58,58,72,5,0,0,62.666667,193,55.619597,0
111,0,1,72,75,80,0,0,0,75.666667,227,65.417867,0


In [143]:
kmeans_clustering_data[kmeans_clustering_data["Успеваемость"] == 1]

,Пол,Приоритет,Балл по математике,Балл по русскому языку,Балл по выбранному предмету,Балл за индивидуальные достижения,Наличие серебрянной медали,Наличие золотой медали,Средний балл по экзаменам,Сумма баллов,Процент от маскимального балла,Успеваемость
2,0,1,58,69,56,0,0,0,61.000000,183,52.737752,1
3,0,1,39,58,68,0,0,0,55.000000,165,47.550432,1
4,0,1,46,52,53,0,0,0,50.333333,151,43.515850,1
5,0,1,52,46,51,0,0,0,49.666667,149,42.939481,1
6,0,1,52,51,61,0,0,0,54.666667,164,47.262248,1
94,0,1,58,72,54,0,0,0,61.333333,184,53.025937,1
95,0,1,60,78,39,0,0,0,59.000000,177,51.008646,1
96,0,1,46,67,64,0,0,0,59.000000,177,51.008646,1
97,1,1,64,61,51,0,0,0,58.666667,176,50.720461,1
98,0,1,52,57,62,0,0,0,57.000000,171,49.279539,1


In [144]:
kmeans_clustering_data[kmeans_clustering_data["Успеваемость"] == 2]

,Пол,Приоритет,Балл по математике,Балл по русскому языку,Балл по выбранному предмету,Балл за индивидуальные достижения,Наличие серебрянной медали,Наличие золотой медали,Средний балл по экзаменам,Сумма баллов,Процент от маскимального балла,Успеваемость
0,1,1,95,67,72,20,0,0,78.000000,254,73.198847,2
7,1,1,94,100,71,10,0,0,88.333333,275,79.250720,2
8,0,1,86,89,85,12,0,0,86.666667,272,78.386167,2
9,1,1,90,89,80,10,0,0,86.333333,269,77.521614,2
10,1,1,88,91,75,30,0,0,84.666667,284,81.844380,2
11,0,1,80,89,80,25,0,0,83.000000,274,78.962536,2
12,0,1,90,83,83,1,0,0,85.333333,257,74.063401,2
13,0,1,70,83,92,10,0,0,81.666667,255,73.487032,2
14,0,1,86,75,83,10,0,0,81.333333,254,73.198847,2
15,0,1,82,83,79,10,0,1,81.333333,254,73.198847,2


In [145]:
kmeans_clustering_data[kmeans_clustering_data["Успеваемость"] == 3]

,Пол,Приоритет,Балл по математике,Балл по русскому языку,Балл по выбранному предмету,Балл за индивидуальные достижения,Наличие серебрянной медали,Наличие золотой медали,Средний балл по экзаменам,Сумма баллов,Процент от маскимального балла,Успеваемость
110,0,1,0,0,48,0,0,0,16.0,48,13.832853,3


**2.4.1.2. Метод SpectralClustering.**

In [146]:
sc = SpectralClustering(n_clusters=4, assign_labels='discretize', random_state=0).fit(df_features)
sc.labels_

array([1, 0, 0, 1, 0, 1, 3, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 3,
       3, 1, 0, 1, 1, 1, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 3, 0, 1, 0, 3, 3, 2, 3, 2, 1, 2, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 1, 0, 1, 0])

In [147]:
df_sc_target = pd.DataFrame(sc.labels_, columns=["Успеваемость"])
df_sc_target.value_counts()

Успеваемость
0               81
1               23
3                7
2                5
Name: count, dtype: int64

In [148]:
sc_clustering_data = pd.concat([df_features, df_sc_target],axis=1)

In [149]:
sc_clustering_data[sc_clustering_data["Успеваемость"] == 0]

,Пол,Приоритет,Балл по математике,Балл по русскому языку,Балл по выбранному предмету,Балл за индивидуальные достижения,Наличие серебрянной медали,Наличие золотой медали,Средний балл по экзаменам,Сумма баллов,Процент от маскимального балла,Успеваемость
1,0,1,64,67,64,0,0,0,65.000000,195,56.195965,0
2,0,1,58,69,56,0,0,0,61.000000,183,52.737752,0
4,0,1,46,52,53,0,0,0,50.333333,151,43.515850,0
7,1,1,94,100,71,10,0,0,88.333333,275,79.250720,0
10,1,1,88,91,75,30,0,0,84.666667,284,81.844380,0
...,...,...,...,...,...,...,...,...,...,...,...,...
108,0,1,40,42,46,0,0,0,42.666667,128,36.887608,0
110,0,1,0,0,48,0,0,0,16.000000,48,13.832853,0
111,0,1,72,75,80,0,0,0,75.666667,227,65.417867,0
113,1,1,70,60,53,4,0,0,61.000000,187,53.890490,0


In [150]:
sc_clustering_data[sc_clustering_data["Успеваемость"] == 1]

,Пол,Приоритет,Балл по математике,Балл по русскому языку,Балл по выбранному предмету,Балл за индивидуальные достижения,Наличие серебрянной медали,Наличие золотой медали,Средний балл по экзаменам,Сумма баллов,Процент от маскимального балла,Успеваемость
0,1,1,95,67,72,20,0,0,78.000000,254,73.198847,1
3,0,1,39,58,68,0,0,0,55.000000,165,47.550432,1
5,0,1,52,46,51,0,0,0,49.666667,149,42.939481,1
8,0,1,86,89,85,12,0,0,86.666667,272,78.386167,1
9,1,1,90,89,80,10,0,0,86.333333,269,77.521614,1
14,0,1,86,75,83,10,0,0,81.333333,254,73.198847,1
20,0,1,80,86,83,0,0,0,83.000000,249,71.757925,1
23,1,1,78,78,82,10,0,0,79.333333,248,71.469741,1
25,0,1,72,83,83,10,0,0,79.333333,248,71.469741,1
26,0,1,88,83,75,0,0,0,82.000000,246,70.893372,1


In [151]:
sc_clustering_data[sc_clustering_data["Успеваемость"] == 2]

,Пол,Приоритет,Балл по математике,Балл по русскому языку,Балл по выбранному предмету,Балл за индивидуальные достижения,Наличие серебрянной медали,Наличие золотой медали,Средний балл по экзаменам,Сумма баллов,Процент от маскимального балла,Успеваемость
28,0,1,82,72,79,10,1,0,77.666667,243,70.028818,2
30,0,1,80,73,82,5,0,0,78.333333,240,69.164265,2
97,1,1,64,61,51,0,0,0,58.666667,176,50.720461,2
99,0,1,64,60,44,0,0,0,56.000000,168,48.414986,2
101,0,1,58,57,46,0,0,0,53.666667,161,46.397695,2


In [152]:
sc_clustering_data[sc_clustering_data["Успеваемость"] == 3]

,Пол,Приоритет,Балл по математике,Балл по русскому языку,Балл по выбранному предмету,Балл за индивидуальные достижения,Наличие серебрянной медали,Наличие золотой медали,Средний балл по экзаменам,Сумма баллов,Процент от маскимального балла,Успеваемость
6,0,1,52,51,61,0,0,0,54.666667,164,47.262248,3
21,0,1,82,89,67,21,0,0,79.333333,259,74.639769,3
22,0,1,82,73,88,5,0,0,81.000000,248,71.469741,3
91,0,1,58,94,62,0,0,0,71.333333,214,61.671470,3
95,0,1,60,78,39,0,0,0,59.000000,177,51.008646,3
96,0,1,46,67,64,0,0,0,59.000000,177,51.008646,3
98,0,1,52,57,62,0,0,0,57.000000,171,49.279539,3


**2.4.1.3. Метод AgglomerativeClustering.**

In [153]:
ac = AgglomerativeClustering(n_clusters=4).fit(df_features)
ac.labels_

array([2, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       3, 1, 0, 0, 0, 0])

In [154]:
df_ac_target = pd.DataFrame(ac.labels_, columns=["Успеваемость"])
df_ac_target.value_counts()

Успеваемость
1               65
0               28
2               22
3                1
Name: count, dtype: int64

In [155]:
ac_clustering_data = pd.concat([df_features, df_ac_target],axis=1)

In [156]:
ac_clustering_data[ac_clustering_data["Успеваемость"] == 0]

,Пол,Приоритет,Балл по математике,Балл по русскому языку,Балл по выбранному предмету,Балл за индивидуальные достижения,Наличие серебрянной медали,Наличие золотой медали,Средний балл по экзаменам,Сумма баллов,Процент от маскимального балла,Успеваемость
1,0,1,64,67,64,0,0,0,65.000000,195,56.195965,0
2,0,1,58,69,56,0,0,0,61.000000,183,52.737752,0
3,0,1,39,58,68,0,0,0,55.000000,165,47.550432,0
4,0,1,46,52,53,0,0,0,50.333333,151,43.515850,0
5,0,1,52,46,51,0,0,0,49.666667,149,42.939481,0
6,0,1,52,51,61,0,0,0,54.666667,164,47.262248,0
92,1,1,64,64,64,5,0,0,64.000000,197,56.772334,0
93,0,1,58,58,72,5,0,0,62.666667,193,55.619597,0
94,0,1,58,72,54,0,0,0,61.333333,184,53.025937,0
95,0,1,60,78,39,0,0,0,59.000000,177,51.008646,0


In [157]:
ac_clustering_data[ac_clustering_data["Успеваемость"] == 1]

,Пол,Приоритет,Балл по математике,Балл по русскому языку,Балл по выбранному предмету,Балл за индивидуальные достижения,Наличие серебрянной медали,Наличие золотой медали,Средний балл по экзаменам,Сумма баллов,Процент от маскимального балла,Успеваемость
28,0,1,82,72,79,10,1,0,77.666667,243,70.028818,1
29,1,1,82,81,67,10,0,0,76.666667,240,69.164265,1
30,0,1,80,73,82,5,0,0,78.333333,240,69.164265,1
31,0,1,74,70,88,8,0,0,77.333333,240,69.164265,1
32,0,1,70,83,78,5,0,0,77.000000,236,68.011527,1
...,...,...,...,...,...,...,...,...,...,...,...,...
88,0,1,74,61,75,4,0,0,70.000000,214,61.671470,1
89,0,1,72,78,59,5,0,0,69.666667,214,61.671470,1
90,0,1,70,75,64,5,0,0,69.666667,214,61.671470,1
91,0,1,58,94,62,0,0,0,71.333333,214,61.671470,1


In [158]:
ac_clustering_data[ac_clustering_data["Успеваемость"] == 2]

,Пол,Приоритет,Балл по математике,Балл по русскому языку,Балл по выбранному предмету,Балл за индивидуальные достижения,Наличие серебрянной медали,Наличие золотой медали,Средний балл по экзаменам,Сумма баллов,Процент от маскимального балла,Успеваемость
0,1,1,95,67,72,20,0,0,78.000000,254,73.198847,2
7,1,1,94,100,71,10,0,0,88.333333,275,79.250720,2
8,0,1,86,89,85,12,0,0,86.666667,272,78.386167,2
9,1,1,90,89,80,10,0,0,86.333333,269,77.521614,2
10,1,1,88,91,75,30,0,0,84.666667,284,81.844380,2
11,0,1,80,89,80,25,0,0,83.000000,274,78.962536,2
12,0,1,90,83,83,1,0,0,85.333333,257,74.063401,2
13,0,1,70,83,92,10,0,0,81.666667,255,73.487032,2
14,0,1,86,75,83,10,0,0,81.333333,254,73.198847,2
15,0,1,82,83,79,10,0,1,81.333333,254,73.198847,2


In [159]:
ac_clustering_data[ac_clustering_data["Успеваемость"] == 3]

,Пол,Приоритет,Балл по математике,Балл по русскому языку,Балл по выбранному предмету,Балл за индивидуальные достижения,Наличие серебрянной медали,Наличие золотой медали,Средний балл по экзаменам,Сумма баллов,Процент от маскимального балла,Успеваемость
110,0,1,0,0,48,0,0,0,16.0,48,13.832853,3


**2.4.1.4. Метод BisectingKMeans.**

In [160]:
bisecting_kmeans = BisectingKMeans(n_clusters=4).fit(df_features)
bisecting_kmeans.labels_

array([2, 3, 0, 0, 1, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 2,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 3, 0, 0, 1, 1], dtype=int32)

In [162]:
df_bisecting_kmeans_target = pd.DataFrame(sc.labels_, columns=["Успеваемость"])
df_bisecting_kmeans_target.value_counts()

Успеваемость
0               81
1               23
3                7
2                5
Name: count, dtype: int64

In [163]:
bisecting_kmeans_clustering_data = pd.concat([df_features, df_bisecting_kmeans_target],axis=1)

In [164]:
bisecting_kmeans_clustering_data[bisecting_kmeans_clustering_data["Успеваемость"] == 0]

,Пол,Приоритет,Балл по математике,Балл по русскому языку,Балл по выбранному предмету,Балл за индивидуальные достижения,Наличие серебрянной медали,Наличие золотой медали,Средний балл по экзаменам,Сумма баллов,Процент от маскимального балла,Успеваемость
1,0,1,64,67,64,0,0,0,65.000000,195,56.195965,0
2,0,1,58,69,56,0,0,0,61.000000,183,52.737752,0
4,0,1,46,52,53,0,0,0,50.333333,151,43.515850,0
7,1,1,94,100,71,10,0,0,88.333333,275,79.250720,0
10,1,1,88,91,75,30,0,0,84.666667,284,81.844380,0
...,...,...,...,...,...,...,...,...,...,...,...,...
108,0,1,40,42,46,0,0,0,42.666667,128,36.887608,0
110,0,1,0,0,48,0,0,0,16.000000,48,13.832853,0
111,0,1,72,75,80,0,0,0,75.666667,227,65.417867,0
113,1,1,70,60,53,4,0,0,61.000000,187,53.890490,0


In [165]:
bisecting_kmeans_clustering_data[bisecting_kmeans_clustering_data["Успеваемость"] == 1]

,Пол,Приоритет,Балл по математике,Балл по русскому языку,Балл по выбранному предмету,Балл за индивидуальные достижения,Наличие серебрянной медали,Наличие золотой медали,Средний балл по экзаменам,Сумма баллов,Процент от маскимального балла,Успеваемость
0,1,1,95,67,72,20,0,0,78.000000,254,73.198847,1
3,0,1,39,58,68,0,0,0,55.000000,165,47.550432,1
5,0,1,52,46,51,0,0,0,49.666667,149,42.939481,1
8,0,1,86,89,85,12,0,0,86.666667,272,78.386167,1
9,1,1,90,89,80,10,0,0,86.333333,269,77.521614,1
14,0,1,86,75,83,10,0,0,81.333333,254,73.198847,1
20,0,1,80,86,83,0,0,0,83.000000,249,71.757925,1
23,1,1,78,78,82,10,0,0,79.333333,248,71.469741,1
25,0,1,72,83,83,10,0,0,79.333333,248,71.469741,1
26,0,1,88,83,75,0,0,0,82.000000,246,70.893372,1


In [166]:
bisecting_kmeans_clustering_data[bisecting_kmeans_clustering_data["Успеваемость"] == 2]

,Пол,Приоритет,Балл по математике,Балл по русскому языку,Балл по выбранному предмету,Балл за индивидуальные достижения,Наличие серебрянной медали,Наличие золотой медали,Средний балл по экзаменам,Сумма баллов,Процент от маскимального балла,Успеваемость
28,0,1,82,72,79,10,1,0,77.666667,243,70.028818,2
30,0,1,80,73,82,5,0,0,78.333333,240,69.164265,2
97,1,1,64,61,51,0,0,0,58.666667,176,50.720461,2
99,0,1,64,60,44,0,0,0,56.000000,168,48.414986,2
101,0,1,58,57,46,0,0,0,53.666667,161,46.397695,2


In [167]:
bisecting_kmeans_clustering_data[bisecting_kmeans_clustering_data["Успеваемость"] == 3]

,Пол,Приоритет,Балл по математике,Балл по русскому языку,Балл по выбранному предмету,Балл за индивидуальные достижения,Наличие серебрянной медали,Наличие золотой медали,Средний балл по экзаменам,Сумма баллов,Процент от маскимального балла,Успеваемость
6,0,1,52,51,61,0,0,0,54.666667,164,47.262248,3
21,0,1,82,89,67,21,0,0,79.333333,259,74.639769,3
22,0,1,82,73,88,5,0,0,81.000000,248,71.469741,3
91,0,1,58,94,62,0,0,0,71.333333,214,61.671470,3
95,0,1,60,78,39,0,0,0,59.000000,177,51.008646,3
96,0,1,46,67,64,0,0,0,59.000000,177,51.008646,3
98,0,1,52,57,62,0,0,0,57.000000,171,49.279539,3


**2.4.1.4. Выбор метода.**

Можно заметить, что кластеры, полученные в результате применения метода AgglomerationClustering, являются наиболее однородными и соответствующими действительности, чем все остальные кластеры, полученные с помощью других методов. Поэтому принято решение использовать данный метод (только необходимо изменить метки классов в соответствии с файлов README.md)

In [172]:
# Изменение меток классов: 3 -> 0 ("Неудовлетворительно"), 2 -> 3 ("Отлично"), 1 -> 2 ("Хорошо"), 0 -> 1 ("Удовлетворительно").
idx = [3,0,1,2]
lut = np.zeros_like(idx)
lut[idx] = np.arange(4)
corrected_target = lut[df_ac_target]

df_target = pd.DataFrame(corrected_labels, columns=["Успеваемость"])

**2.4.2. Генерация новых данных.**

**2.4.2.1. Метод OriginalGenerator.**

In [173]:
extended_df_features, extended_df_target = OriginalGenerator().generate_data_pipe(df_features.drop(columns=["Средний балл по экзаменам", "Сумма баллов", "Процент от маскимального балла"]), df_target, df_features.drop(columns=["Средний балл по экзаменам", "Сумма баллов", "Процент от маскимального балла"]))

In [174]:
extended_df_features.head()

,Пол,Приоритет,Балл по математике,Балл по русскому языку,Балл по выбранному предмету,Балл за индивидуальные достижения,Наличие серебрянной медали,Наличие золотой медали
0,0,1,46,67,64,0,0,0
1,0,1,46,67,64,0,0,0
2,0,1,46,67,64,0,0,0
3,0,1,46,67,64,0,0,0
4,0,1,52,51,61,0,0,0


In [175]:
extended_df_features.describe()

,Пол,Приоритет,Балл по математике,Балл по русскому языку,Балл по выбранному предмету,Балл за индивидуальные достижения,Наличие серебрянной медали,Наличие золотой медали
count,331.000000,331.000000,331.000000,331.000000,331.000000,331.000000,331.000000,331.000000
mean,0.145015,1.241692,68.864048,70.673716,68.287009,4.383686,0.039275,0.012085
std,0.352649,0.481993,12.994740,11.068487,11.215484,5.572191,0.194542,0.109429
min,0.000000,1.000000,39.000000,42.000000,39.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,64.000000,64.000000,62.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,72.000000,72.000000,70.000000,1.000000,0.000000,0.000000
75%,0.000000,1.000000,78.000000,78.000000,77.000000,10.000000,0.000000,0.000000
max,1.000000,3.000000,90.000000,97.000000,92.000000,25.000000,1.000000,1.000000


In [176]:
extended_df_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331 entries, 0 to 330
Data columns (total 8 columns):
 #   Column                             Non-Null Count  Dtype
---  ------                             --------------  -----
 0   Пол                                331 non-null    int64
 1   Приоритет                          331 non-null    int64
 2   Балл по математике                 331 non-null    int64
 3   Балл по русскому языку             331 non-null    int64
 4   Балл по выбранному предмету        331 non-null    int64
 5   Балл за индивидуальные достижения  331 non-null    int64
 6   Наличие серебрянной медали         331 non-null    int64
 7   Наличие золотой медали             331 non-null    int64
dtypes: int64(8)
memory usage: 20.8 KB


In [177]:
extended_df_target.value_counts()

Успеваемость
2    192
1     94
3     45
Name: count, dtype: int64

**2.4.2.2. Метод GANGenerator.**

In [178]:
extended_df_features, extended_df_target = GANGenerator(gen_x_times=100, cat_cols=["Пол", "Приоритет", "Наличие серебрянной медали", "Наличие золотой медали"], adversarial_model_params={'learning_rate': 0.02, 'random_state': 42}, gen_params={'batch_size': 500, 'epochs': 40}).generate_data_pipe(df_features.drop(columns=["Средний балл по экзаменам", "Сумма баллов", "Процент от маскимального балла"]), df_target, df_features.drop(columns=["Средний балл по экзаменам", "Сумма баллов", "Процент от маскимального балла"]))

Fitting CTGAN transformers for each column:   0%|          | 0/9 [00:00<?, ?it/s]

Training CTGAN, epochs::   0%|          | 0/40 [00:00<?, ?it/s]

In [179]:
extended_df_features.head()

,Пол,Приоритет,Балл по математике,Балл по русскому языку,Балл по выбранному предмету,Балл за индивидуальные достижения,Наличие серебрянной медали,Наличие золотой медали
0,0,1,86,50,82,3,0,0
1,0,1,86,49,83,3,0,0
2,1,1,86,42,81,2,0,0
3,0,1,86,45,80,2,0,0
4,0,1,85,43,79,2,0,0


In [180]:
extended_df_features.head()

,Пол,Приоритет,Балл по математике,Балл по русскому языку,Балл по выбранному предмету,Балл за индивидуальные достижения,Наличие серебрянной медали,Наличие золотой медали
0,0,1,86,50,82,3,0,0
1,0,1,86,49,83,3,0,0
2,1,1,86,42,81,2,0,0
3,0,1,86,45,80,2,0,0
4,0,1,85,43,79,2,0,0


In [181]:
extended_df_features.describe()

,Пол,Приоритет,Балл по математике,Балл по русскому языку,Балл по выбранному предмету,Балл за индивидуальные достижения,Наличие серебрянной медали,Наличие золотой медали
count,13600.000000,13600.000000,13600.000000,13600.000000,13600.000000,13600.000000,13600.000000,13600.000000
mean,0.186324,1.285074,78.352132,63.076250,76.711912,4.221471,0.087574,0.065147
std,0.389382,0.564520,11.894846,14.510378,11.354729,5.087823,0.282684,0.246794
min,0.000000,1.000000,2.000000,20.000000,22.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,73.000000,52.000000,70.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,80.000000,62.000000,79.000000,2.000000,0.000000,0.000000
75%,0.000000,1.000000,87.000000,73.000000,85.000000,7.000000,0.000000,0.000000
max,1.000000,3.000000,94.000000,99.000000,93.000000,29.000000,1.000000,1.000000


In [182]:
extended_df_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13600 entries, 0 to 13599
Data columns (total 8 columns):
 #   Column                             Non-Null Count  Dtype
---  ------                             --------------  -----
 0   Пол                                13600 non-null  int64
 1   Приоритет                          13600 non-null  int64
 2   Балл по математике                 13600 non-null  int64
 3   Балл по русскому языку             13600 non-null  int64
 4   Балл по выбранному предмету        13600 non-null  int64
 5   Балл за индивидуальные достижения  13600 non-null  int64
 6   Наличие серебрянной медали         13600 non-null  int64
 7   Наличие золотой медали             13600 non-null  int64
dtypes: int64(8)
memory usage: 850.1 KB


In [183]:
extended_df_target.value_counts()

Успеваемость
 2    8533
 1    3656
 3     826
 0     463
 4      99
-1      21
-2       2
Name: count, dtype: int64

**2.4.2.3. Метод ForestDifussionGenerator.**

In [184]:
extended_df_features, extended_df_target = ForestDiffusionGenerator(gen_x_times=100, cat_cols=["Пол", "Приоритет", "Наличие серебрянной медали", "Наличие золотой медали"], adversarial_model_params={'learning_rate': 0.02, 'random_state': 42}, gen_params={'batch_size': 500, 'epochs': 40}).generate_data_pipe(df_features.drop(columns=["Средний балл по экзаменам", "Сумма баллов", "Процент от маскимального балла"]), df_target, df_features.drop(columns=["Средний балл по экзаменам", "Сумма баллов", "Процент от маскимального балла"]))

In [185]:
extended_df_features.head()

,Пол,Приоритет,Балл по математике,Балл по русскому языку,Балл по выбранному предмету,Балл за индивидуальные достижения,Наличие серебрянной медали,Наличие золотой медали
0,0,1,75,66,74,2,0,0
1,0,1,75,66,73,2,0,0
2,0,1,75,67,73,2,0,0
3,0,1,75,77,64,9,0,0
4,0,1,75,66,74,0,0,0


In [186]:
extended_df_features.describe()

,Пол,Приоритет,Балл по математике,Балл по русскому языку,Балл по выбранному предмету,Балл за индивидуальные достижения,Наличие серебрянной медали,Наличие золотой медали
count,11480.000000,11480.000000,11480.000000,11480.000000,11480.000000,11480.000000,11480.000000,11480.000000
mean,0.106620,1.147648,70.304355,71.145645,68.609756,4.664721,0.010279,0.010801
std,0.308643,0.382647,12.277998,10.859037,10.971836,5.768545,0.100866,0.103371
min,0.000000,1.000000,2.000000,5.000000,22.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,65.000000,66.000000,62.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,74.000000,72.000000,71.000000,3.000000,0.000000,0.000000
75%,0.000000,1.000000,78.000000,78.000000,77.000000,9.000000,0.000000,0.000000
max,1.000000,3.000000,94.000000,99.000000,93.000000,29.000000,1.000000,1.000000


In [187]:
extended_df_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11480 entries, 0 to 11479
Data columns (total 8 columns):
 #   Column                             Non-Null Count  Dtype
---  ------                             --------------  -----
 0   Пол                                11480 non-null  int64
 1   Приоритет                          11480 non-null  int64
 2   Балл по математике                 11480 non-null  int64
 3   Балл по русскому языку             11480 non-null  int64
 4   Балл по выбранному предмету        11480 non-null  int64
 5   Балл за индивидуальные достижения  11480 non-null  int64
 6   Наличие серебрянной медали         11480 non-null  int64
 7   Наличие золотой медали             11480 non-null  int64
dtypes: int64(8)
memory usage: 717.6 KB


In [188]:
extended_df_target.value_counts()

Успеваемость
2    4698
1    4481
0    1253
3    1048
Name: count, dtype: int64

**2.4.2.3. Выбор метода.**

Можно заметить, что первый метод просто дублирует объекты, при этом в сгенерированных данных даже нет объектов класса 0, второй метод хоть и создаёт разнообразные новые объекты, но определяет некоторые из них новыми классами, которые нас не интересуют. В то время третий метод создаёт разнообразные новые объекты и при этом определяет все из них только интересующими нас классами, которые были в исходных данных. Поэтому для генерации новых данных будем использовать метод ForestDiffusionGenerator.